# Defocus estimation using DenseNet 201

DenseNet 201 is a CNN model developed by Cornell University, Tsinghua University, and Facebook AI Research in 2017.

Keras documentation:
[https://keras.io/api/applications/densenet/#densenet201-function](https://keras.io/api/applications/densenet/#densenet201-function)

The original paper:
[https://arxiv.org/pdf/1608.06993.pdf](https://arxiv.org/pdf/1608.06993.pdf)

In [ ]:
"""
Enables HiDPI rendering of matplotlib charts on Apple Retina displays.
Comment out the line below if the matplotlib plots don’t look right on your computer.
"""
%config InlineBackend.figure_format = "retina"

# Library imports
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from dataset import Dataset
from plot import Plot
from mip import Microscopy_image_processor

In [ ]:
# Download and extract the dataset if it’s not present.

local_data_folder = os.path.join(os.path.abspath(os.getcwd()), "data")
dataset_ulr = "https://www.robots.ox.ac.uk/~vgg/data/dtd/download/dtd-r1.0.1.tar.gz"

Dataset.download_if_missing(local_data_folder, dataset_ulr)

In [ ]:
# Prepare the data

import os
import random
from dataset import Dataset

sample_side_size = 224
training_set_percentage = 0.2

all_features, all_labels = Dataset.get_features_and_labels(local_data_folder, sample_side_size)

no_of_test_items = int(round(all_features.shape[0] * training_set_percentage))

training_features = all_features[:-no_of_test_items]
test_features = all_features[-no_of_test_items:]

training_labels = all_labels[:-no_of_test_items]
test_labels = all_labels[-no_of_test_items:]

print(f"all_features.shape: {all_features.shape}")
print(f"labels.shape: {all_labels.shape}")

print(f"training_features.shape: {training_features.shape}")
print(f"training_labels.shape: {training_labels.shape}")

print(f"test_features.shape: {test_features.shape}")
print(f"test_labels.shape: {test_labels.shape}")

In [ ]:
# Prepare the model: DenseNet201

import keras
from keras.applications import DenseNet201
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

# Create the base pre-trained model
base_model = DenseNet201(weights = None, include_top = False)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# Let’s add a fully-connected layer
x = Dense(1024, activation="relu")(x)
# And a logistic layer
predictions = Dense(1, activation="sigmoid")(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer="rmsprop", loss="mean_absolute_error")

model.summary()

In [ ]:
# Train the model on the new data for a few epochs

batch_size = 32
epochs = 1

history = model.fit(training_features, training_labels, validation_split = 0.33, epochs=epochs, batch_size = batch_size, verbose = 1)

In [ ]:
# Plot training history

metrics = history.history
plt.figure(figsize=(11,3))
plt.subplot(1,2,1)
plt.plot(history.epoch, metrics["loss"], metrics["val_loss"])
plt.legend(["Loss (Training)", "Loss (Cross-validation)"])
plt.ylim([0, max(plt.ylim())])
plt.xlabel("Epoch")
plt.ylabel("Loss (Mean absolute error)")
plt.show()

In [ ]:
# Evaluate the trained model against the test set

model.evaluate(x = test_features, y = test_labels)

In [ ]:
# Evaluate the trained model against three manually selected files 

p_files = ["zigzagged_0014.jpg", "woven_0045.jpg", "veined_0150.jpg"]
p_weights = [0.9, 0.8, 0.7]
p_prefix = "densenet"

p_folder = os.path.join(os.path.abspath(os.getcwd()), "predictions")

# Change to True to save the images
save_images = False

if save_images and not os.path.exists(p_folder):
    os.mkdir(p_folder)

for i, file in enumerate(p_files):
    p_file_full_path = os.path.join(local_data_folder, file)
    print(f"Image {i+1}: {p_file_full_path}\n")
    p_base = Dataset.get_single_sample(p_file_full_path, 0, sample_side_size)[0]
    print("Base image:")
    if save_images:
        Microscopy_image_processor.save_image(os.path.join(p_folder, f"{p_prefix}_img{i+1}_base.jpg"), p_base)
    Plot.normalized_mono_image(p_base)

    print(f"Aberrated image. Actual weight: {p_weights[i]}")
    p_wrapped_sample = Dataset.get_single_sample(p_file_full_path, p_weights[i], sample_side_size)
    p_sample = p_wrapped_sample[0]
    if save_images:
        Microscopy_image_processor.save_image(os.path.join(p_folder, f"{p_prefix}_img{i+1}_abberated.jpg"), p_sample)
    Plot.normalized_mono_image(p_sample)

    p_estimate = model.predict(p_wrapped_sample)[0][0]
    p_delta = abs(p_weights[i] - p_estimate)
    print(f"Simulated focus correction. Absolute error: {p_delta}")
    p_corrected = Dataset.get_single_sample(p_file_full_path, p_delta, sample_side_size)[0]
    if save_images:
        Microscopy_image_processor.save_image(os.path.join(p_folder, f"{p_prefix}_img{i+1}_corrected.jpg"), p_corrected)
    Plot.normalized_mono_image(p_corrected)

In [ ]:
# Absolute error of weight predictions

for i, file in enumerate(p_files):
    p_file_full_path = os.path.join(local_data_folder, file)
    print(f"Image {i+1}: {p_file_full_path}\n")
    steps = []
    errors = []
    for step in np.linspace(0, 1, 21):
        p_wrapped_sample = Dataset.get_single_sample(p_file_full_path, step, sample_side_size)
        p_sample = p_wrapped_sample[0]
        p_estimate = model.predict(p_wrapped_sample)[0][0]
        steps.append(round(step, 5))
        errors.append(abs(step - p_estimate))

    plt.figure(figsize=(11, 3))
    plt.subplot(1, 2, 1)
    plt.xticks(np.arange(0, 1.2, 0.2))
    plt.plot(steps, errors)
    plt.legend([f"Image {i+1}"])
    plt.ylim([0, 1])
    plt.xlabel("Actual defocus weight")
    plt.ylabel("Absolute error")
    plt.show()

In [ ]:
# Simulated re-focus behaviour of a microscope

for i, file in enumerate(p_files):
    p_file_full_path = os.path.join(local_data_folder, file)
    print(f"Image {i+1}: {p_file_full_path}\n")
    print(f"Initial defocus weight: {p_weights[i]}")
    last_weight = p_weights[i]
    steps = []
    estimates = []
    errors = []
    for step in range(11):
        p_wrapped_sample = Dataset.get_single_sample(p_file_full_path, last_weight, sample_side_size)
        p_sample = p_wrapped_sample[0]
        steps.append(step)
        errors.append(last_weight)
        p_estimate = model.predict(p_wrapped_sample)[0][0]
        estimates.append(p_estimate)
        print(f"Defocus estimate: {p_estimate}")
        last_weight = abs(last_weight - p_estimate)
        print(f"New defocus weight: {last_weight}")

    plt.figure(figsize=(11, 3))
    plt.subplot(1, 2, 1)
    plt.xticks(np.arange(0, 12, 1))
    plt.plot(steps, errors, estimates)
    plt.legend(["Actual defocus weight", "Estimated defocus weight"])
    plt.ylim([0, 1])
    plt.xlabel("Refocus step")
    plt.ylabel("Absolute error")
    plt.show()